## Neste tutorial, você aprenderá como detectar automaticamente o COVID-19 em um conjunto de dados de imagem de raio-X criado à mão usando Keras, TensorFlow e Deep Learning.

## Simplificando: você não precisa de um diploma em medicina para causar impacto no campo da medicina  - profissionais de aprendizado profundo que trabalham em estreita colaboração com médicos e profissionais da área médica podem resolver problemas complexos, salvar vidas e tornar o mundo um lugar melhor.

## Como o COVID-19 pode ser detectado em imagens de raios-X?

In [1]:
import pandas as pd
import random
import shutil
import os

In [ ]:
# !pip install pillow

In [2]:
def getDiretorio(tipo):
    retorno = ""
    if(tipo =="MaquinaLocal"):
        retorno = r'D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto'
    
    if(tipo =="GitHub"):
        retorno = r'https://github.com/ieee8023/covid-chestxray-dataset/blob/master/'

    if(tipo =="GoogleDrive"):
       retorno=r'/content/drive/My Drive/TCC Pos Graduação big data/TCC/Projeto/'
       
    
    return retorno
    

In [5]:
diretorio_raiz = getDiretorio("MaquinaLocal")
diretorio_dataset =r"{0}/{1}".format(diretorio_raiz,'dataset')
diretorio_imagem_covid =r"{0}/{1}/{2}".format(diretorio_dataset,'covid','images')
diretorio_pneumonia_normal =r"{0}/{1}".format(diretorio_dataset,'normal')
diretorio_imagens_notebook =r"{0}/{1}".format(diretorio_raiz,'Imagens_notebook')
diretorio_save_models =r"{0}/{1}".format(diretorio_raiz,'models')
diretorio_save_model_imagens =r"{0}/{1}".format(diretorio_save_models,'imagens')
print(diretorio_raiz)
print(diretorio_dataset)
print(diretorio_imagem_covid)
print(diretorio_pneumonia_normal)
print(diretorio_imagens_notebook)
print(diretorio_save_models)
print(diretorio_save_model_imagens)



D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/covid/images
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/normal
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/Imagens_notebook
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/models
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/models/imagens


In [23]:
source_dir_metadata = r'D:/Repos/datasetsGitHub/covid-chestxray-dataset/metadata.csv'
source_dir_kaggle_Covid =r'D:/Repos/datasetsGitHub/covid-chestxray-dataset/images'
source_dir_kaggle_pneumonia =r'D:/Repos/pneumonia-chest_xray'






In [24]:
if not os.path.exists(diretorio_imagem_covid):
    os.mkdir(diretorio_imagem_covid)

if not os.path.exists(diretorio_pneumonia_normal):
    os.mkdir(diretorio_pneumonia_normal)


In [25]:
df  = pd.read_csv(source_dir_metadata)

In [10]:
df.shape

(565, 28)

In [9]:
df.head(2)

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,in_icu,...,modality,date,location,folder,filename,doi,url,license,clinical_notes,other_notes
0,2,0.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,X-ray,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN
1,2,3.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,X-ray,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN


PA View of X ray

In [11]:
labels =[]
lista_imagens_erro = []
lista_imagens_com_Covid = []
lista_imagens_com_pneumonia_normal = []
lista_imagens_com_pneumonia_pneumonia = []
lista_tuplas_imagens =[]

## O conjunto de dados de imagem de raios-X COVID-19 que usaremos para este tutorial foi curado pelo Dr. Joseph Cohen , um pós-doutorado na Universidade de Montreal
https://josephpcohen.com/w/

## git hub dos dados
https://josephpcohen.com/w/public-covid19-dataset/

https://github.com/ieee8023/covid-chestxray-dataset

In [19]:
lista_imagens_com_Covid =[]

quantidade = 0
for (i,row) in df.iterrows():
    if row["finding"] =="COVID-19" and  row["view"] =="PA":
        file_name=row["filename"]
        image_path  = "{0}/{1}".format(source_dir_imagens_Covid,file_name)
        image_copy_target= "{0}/{1}".format(diretorio_imagem_covid,file_name)
        shutil.copy2(image_path,image_copy_target)
        lista_imagens_com_Covid.append(image_copy_target)
        quantidade +=1

len((lista_imagens_com_Covid))

182

In [14]:
lista_imagens_com_Covid[0:2]

['D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/covid/images/auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg',
 'D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/covid/images/auntminnie-b-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg']

## recuperar imagens do diretorio de pneumonia


In [15]:
imagens_normal = os.listdir("{0}/{1}/{2}".format(source_dir_pneumonia,"test","NORMAL"))



In [41]:
for i in range(len(lista_imagens_com_Covid)):
    file_name = imagens_normal[i]
    image_path  = "{0}/{1}/{2}/{3}".format(source_dir_pneumonia,"test","NORMAL",file_name)
    image_copy_target= "{0}/{1}".format(diretorio_pneumonia_normal,file_name)
    shutil.copy2(image_path,image_copy_target)
    lista_imagens_com_pneumonia_normal.append(image_copy_target)
    # quantidade +=1
    
    

In [ ]:

from IPython.display import Image
Image('{0}/{1}'.format(diretorio_imagens_notebook,'imagensVariosRx.PNG'))

In [43]:
class ConfiguracoesRedeNeuralCovid(object):
    def __init__(self):
        self.IdModelo =""
        self.NomeModelo=""
        self.TamanhoModelo=""
        self.EPOCHS=""
        self.INIT_LR=""
        self.BS=""
        self.IMAGE_SIZE=""
        self.Descricao=""
        self.DataCriacao=""
        self.QuantidadeImagensTotais=""
        self.QuantidadeImagensCovid=""
        self.QuantidadeImagensPneumonia=""
        self.QuantidadeImagensNormal =""
        self.QuantidadeImagensTrainX=""
        self.QuantidadeImagensTestX=""
        self.QuantidadeImagensTrainY=""
        self.QuantidadeImagensTestY=""
        self.Rotation_range=""
        self.Fill_mode=""
        self.Test_size=""
        self.TipoCNN=""
        self.NomeNoteBook = ""

    def to_dict(self):
        return {
            'IdModelo' : self.IdModelo,
            'NomeModelo' :self.NomeModelo,
            'TamanhoModelo':self.TamanhoModelo,
            'EPOCHS' :self.EPOCHS,
            'INIT_LR' :self.INIT_LR,
            'BS' : self.BS,
            'IMAGE_SIZE' :self.IMAGE_SIZE,
            'Descricao' :self.Descricao,
            'DataCriacao' :self.DataCriacao,
            'QuantidadeImagensTotais' :self.QuantidadeImagensTotais,
            'QuantidadeImagensCovid' :self.QuantidadeImagensCovid,
            'QuantidadeImagensPneumonia' :self.QuantidadeImagensPneumonia,
            'QuantidadeImagensNormal' :self.QuantidadeImagensNormal,
            'QuantidadeImagensTrainX' :self.QuantidadeImagensTrainX,
            'QuantidadeImagensTestX' :self.QuantidadeImagensTestX,
            'QuantidadeImagensTrainY' :self.QuantidadeImagensTrainY,
            'QuantidadeImagensTestY' :self.QuantidadeImagensTestY,
            'Rotation_range' :self.Rotation_range,
            'fill_mode' :self.Fill_mode,
            'test_size' :self.Test_size,
            "TipoCNN" :self.TipoCNN,
            "NomeNoteBook" :self.NomeNoteBook

        }
    # Métodos são funções, que recebem como parâmetro atributos do objeto criado    
    def getInformacoes(self):
        print('Modelo {0}|{1} criado em {2} |com {3} EPOCHS|{4} IMG_COVID| {5} IMGPNEU|{6} IMGNORMAL'.
        format(self.IdModelo,self.NomeModelo,str(self.DataCriacao),self.EPOCHS,self.QuantidadeImagensCovid,self.QuantidadeImagensPneumonia,self.QuantidadeImagensNormal))

In [ ]:
config= ConfiguracoesRedeNeuralCovid()
config.DataCriacao = datetime.now()
config.IMAGE_SIZE = (224,224)
config.INIT_LR = 1e-3
config.EPOCHS = 100
config.BS = 8
config.IdModelo = uuid.uuid1()
config.Rotation_range=25
config.Fill_mode= str("nearest")
config.Test_size=0.30
config.TipoCNN ="VGG16"
config.NomeNoteBook = "Artigo_V2"

config.NomeModelo ="Model_{0}_{1}_{2}_EPOCHS".format(config.TipoCNN,config.NomeNoteBook,config.EPOCHS) 


In [67]:
urlbase = getDiretorio('GitHub')
urlmetadata = "{}/{}".format(urlbase,'metadata.csv')
metadata_git = pd.read_csv('https://raw.githubusercontent.com/ieee8023/covid-chestxray-dataset/master/metadata.csv',error_bad_lines=False)


metadata_local   = pd.read_csv(diretorio_dataset +'/covid/metadata.csv')


In [61]:
metadata_local.head(1)

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,in_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 28
0,2,0.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN


In [80]:
df_metadata_git =  metadata_git[["patientid","sex","age","location", "finding", "modality","filename","view"]]

df_metadata_git.columns = ["patientId","sexo","idade","localidade","prognostico","tipoExame","imagem","posicao_imagem"]

df_metadata_local=  metadata_local[["patientid","sex","age","location", "finding", "modality","filename","view"]]

df_metadata_local.columns = ["patientId","sexo","idade","localidade","prognostico","tipoExame","imagem","posicao_imagem"]



## Dentro do repositório, você encontrará exemplos de casos COVID-19, além de MERS, SARS e ARDS.

In [64]:
df_metadata_local["prognostico"].value_counts()

COVID-19          504
Streptococcus      17
No Finding         17
SARS               16
Pneumocystis       15
COVID-19, ARDS     12
ARDS                4
E.Coli              4
Chlamydophila       2
Legionella          2
Klebsiella          1
Name: prognostico, dtype: int64

In [72]:
df_metadata_local["posicao_imagem"].value_counts()

PA               247
AP               188
Axial             68
AP Supine         45
L                 29
Coronal           16
AP semi erect      1
Name: posicao_imagem, dtype: int64

In [70]:
prognostico_map_dataframe ={
    "COVID-19" : 1,
    "COVID-19, ARDS":2,
    "Streptococcus" :3,
    "SARS" :4,
    "Pneumocystis" :5,
    "E.Coli" :6,
    "No Finding" :7,
    "Chlamydophila" :8,
    "Legionella" :9,
    "Klebsiella" :10
}

In [81]:
df_metadata_local['indicadorPrognostico'] = df_metadata_local['prognostico'].map(prognostico_map_dataframe)
df_metadata_git['indicadorPrognostico'] = df_metadata_git['prognostico'].map(prognostico_map_dataframe)


df_metadata_local.query('indicadorPrognostico == 1 and posicao_imagem =="PA" ',inplace = True)

df_metadata_local["prognostico"].replace(["COVID-19"],["covid"],inplace=True)

df_metadata_git.query('indicadorPrognostico == 1 and posicao_imagem =="PA" ',inplace=True)

df_metadata_git["prognostico"].replace(["COVID-19"],["covid"],inplace=True)

df_metadata_git["prognostico"].replace(["COVID-19"],["covid"],inplace=True)

print(len(df_metadata_local))
print(len(df_metadata_git))

183
182


In [ ]:

# df_metadata_local.query('indicadorPrognostico = 1 and posicao_imagem ="PA"',inplace=True)

In [82]:
AtualizarBancoDeDados = False

print(len(df_metadata_git))
print(len(df_metadata_local))

if(len(df_metadata_git) > len(df_metadata_local) ):
    AtualizarBancoDeDados = True

if(AtualizarBancoDeDados):
  print('Já podemos atualizar o banco de imagens  temos {0} imagens com covid a mais'.format(len(df_metadata_git) - len(df_metadata_local)))


182
183


## recuperar as imagens com covid

In [84]:
diagnostico_map= {'pneumonia' : 2, 'covid' : 1, 'normal' : 0}

In [86]:
labels =[]
lista_imagens_erro = []
lista_imagens_com_Covid = []
lista_imagens_com_pneumonia_normal = []
lista_imagens_com_pneumonia_pneumonia = []
lista_tuplas_imagens =[]

## importando os pacotes 

In [89]:
import numpy as np
import keras
from  keras.layers import *
from keras.models import *
from keras.preprocessing import image

## criação da arquitera da rede neural


In [98]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1,activation='sigmoid'))


model.compile(loss = keras.losses.binary_crossentropy,optimizer="adam",metrics=["accuracy"])

## Compilação da rede neural

In [99]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 54, 54, 64)       

## generate train data gem  

In [102]:
train_datagen  = image.ImageDataGenerator(rescale= 1.0/255,shear_range= 0.2 ,zoom_range=0.2,horizontal_flip=True)

test_dataset  = image.ImageDataGenerator(rescale= 1.0/255)

In [104]:
train_generator  = train_datagen.flow_from_dataframe(,
                                                    target_size =(224,224),
                                                    batch_size=32,
                                                    class_mode='binary'
                                                    )

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'CovidDataset/Train'

In [87]:


for row in df_metadata_local[["prognostico","imagem"]].iterrows():
    imagen_name = row[1].imagem
    prognostico = row[1].prognostico
    full_name = r"{0}/{1}".format(diretorio_imagem_covid,imagen_name)
    image = cv2.imread(full_name)
    if(image is None):
        lista_imagens_erro.append(imagen_name)
    else:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, config.IMAGE_SIZE)
        lista_tuplas_imagens.append((prognostico,image))
        lista_imagens_com_Covid.append(full_name)
        
   


NameError: name 'cv2' is not defined

In [ ]:


print("quantidade imagens com covid {0}".format(len(lista_imagens_com_Covid)))

### Para fazer isso, usei o conjunto de dados de imagens de raios X do tórax de Kaggle (Pneumonia) e coletei 275 imagens de raios X de pacientes saudáveis ( Figura 2 , à direita ). Existem vários problemas com o conjunto de dados de radiografia de tórax do Kaggle, ou seja, rótulos ruidosos / incorretos, mas serviu como um ponto de partida suficientemente bom para esta prova de conceito do detector COVID-19.

https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

baixamos os arquivos de test  sem pneumonia


In [ ]:

imagens_pneumonia_normal =  list(paths.list_images(diretorio_pneumonia_normal))
imagens_pneumonia_pneumonia =  list(paths.list_images(diretorio_pneumonia_pneumonia))
print("Imagens Pneumonia Normal {0}".format(len(imagens_pneumonia_normal)))
print("Imagens Pneumonia Pneumonia {0}".format(len(imagens_pneumonia_pneumonia)))

In [ ]:
for item in imagens_pneumonia_normal:
    image = cv2.imread(item)
    prognostico = item.split(os.path.sep)[-2]
    prognostico = str(prognostico).lower()
    if(image is None):
        lista_imagens_erro.append(item)
    else:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, config.IMAGE_SIZE)
        lista_tuplas_imagens.append((prognostico,image))
        lista_imagens_com_pneumonia_normal.append(item)
    

In [ ]:
for item in imagens_pneumonia_pneumonia:
    image = cv2.imread(item)
    prognostico = item.split(os.path.sep)[-2]
    prognostico = str(prognostico).lower()
    if(image is None):
        lista_imagens_erro.append(item)
    else:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, config.IMAGE_SIZE)
        # image = cv2.resize(image, (224, 224), interpolation = cv2.INTER_AREA)
        lista_tuplas_imagens.append((prognostico,image))
        lista_imagens_com_pneumonia_pneumonia.append(item)

In [ ]:
config.QuantidadeImagensCovid =len(lista_imagens_com_Covid)
config.QuantidadeImagensPneumonia =len(lista_imagens_com_pneumonia_pneumonia)
config.QuantidadeImagensNormal =len(lista_imagens_com_pneumonia_normal)
config.QuantidadeImagensTotais = len(lista_tuplas_imagens)

print("Imagens Covid  {0} total de {1:.2f} %".format(len(lista_imagens_com_Covid),len(lista_imagens_com_Covid) / (len(lista_tuplas_imagens)) * 100))
print("Imagens Pneumonia Pneumonia  {0} total de {1:.2f} %".format(len(lista_imagens_com_pneumonia_pneumonia),len(lista_imagens_com_pneumonia_pneumonia) / (len(lista_tuplas_imagens)) * 100))
print("Imagens Pneumonia Normais  {0} total de {1:.2f} %".format(len(lista_imagens_com_pneumonia_normal),len(lista_imagens_com_pneumonia_normal) / (len(lista_tuplas_imagens)) * 100))


print("Total de Imagens {0}".format(len(lista_tuplas_imagens)))
 

In [ ]:
labels = [item[0] for item in lista_tuplas_imagens]



In [ ]:
distinct_labels  = set(labels)

distinct_labels


In [ ]:

labels = np.array(labels)


In [ ]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
# labels = to_categorical(labels, num_classes = 2)


In [ ]:
labels.shape

In [ ]:
labels[0:10]

In [ ]:
lista_imagens =[item[1] for item in lista_tuplas_imagens]

In [ ]:
lista_imagens = np.array(lista_imagens) / 255.0

In [ ]:
lista_imagens.shape


## fazendo a divisão do data-set em test e treino



In [ ]:
config.Test_size = 0.30

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(lista_imagens, labels, train_size = config.Test_size, stratify=labels, random_state=42)

In [ ]:
print("trainX shape {0}".format(trainX[0].shape))
print("testX shape {0}".format(testX[0].shape))
print("trainY shape {0}".format(trainY[0].shape))
print("testY shape {0}".format(testY[0].shape))


config.QuantidadeImagensTestX = len(testX)
config.QuantidadeImagensTestY = len(testY)
config.QuantidadeImagensTrainX = len(trainX)
config.QuantidadeImagensTrainY = len(trainY)
print("\n")
print("quantidade registros em  trainX {0}".format(config.QuantidadeImagensTrainX))
print("quantidade registros em  testX {0}".format(config.QuantidadeImagensTestX))
print("quantidade registros em  trainY {0}".format(config.QuantidadeImagensTrainY))
print("quantidade registros em  testY {0}".format(config.QuantidadeImagensTestY))

## executando a  tecnica de  data augmentation

### Para garantir a generalização do nosso modelo, realizamos o aumento de dados definindo a rotação aleatória da imagem em 15 graus no sentido horário ou anti-horário.

In [ ]:
config.Fill_mode

In [ ]:
trainAug = ImageDataGenerator(rotation_range=config.Rotation_range,fill_mode= config.Fill_mode)


### instanciamentos a rede VGG16 com pesos pré-treinados no ImageNet, deixando  o head da camada FC.

A partir daí, construímos um nova camanda totalmente conectada, consistindo em POOL => FC = SOFTMAX em cima do VGG16 

Em seguida, congelamos o CONV pesos do VGG16, de modo que apenas oFCa cabeça da camada será treinada ( Linhas 101-102 ); isso completa nossa configuração de ajuste fino.

In [ ]:
# carregar a rede VGG16, garantindo que os conjuntos de camadas FC principais sejam deixados
# fora
baseModel = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))
# construa o head  do modelo que será colocado em cima d0 modelo base
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)
# coloque o modelo FC principal sobre o modelo base (isso se tornará
# o modelo real que vamos treinar)
model = Model(inputs=baseModel.input, outputs=headModel)
# passa por todas as camadas no modelo base e congela-as para que elas
# * not * seja atualizado durante o primeiro processo de treinamento
for layer in baseModel.layers:
	layer.trainable = False

### Agora estamos prontos para compilar e treinar nosso modelo de aprendizado

## compilam a rede com o declínio da taxa de aprendizado e osAdãootimizador. Dado que este é um problema de 2 classes, usamos"binary_crossentropy" perda em vez de crossentropia categórica.


In [ ]:
print(float(config.Rotation_range))
print(config.EPOCHS)
print(config.INIT_LR)
print(config.BS)

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=config.INIT_LR, decay=config.INIT_LR / config.EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])


# train the head of the network
print("[INFO] training head...")
H = model.fit_generator(trainAug.flow(trainX, trainY, batch_size=config.BS),steps_per_epoch=len(trainX) // config.BS,
	validation_data=(testX, testY),validation_steps=len(testX) // config.BS,epochs=config.EPOCHS)

## A seguir, avaliaremos nosso modelo:

## primeiro fazemos previsões no conjunto de testes e obtemos os índices de previsão

## Em seguida, geramos e imprimimos um relatório de classificação usando o utilitário auxiliar do scikit-learn


In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=config.BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))
	

## Em seguida, calcularemos uma matriz de confusão para uma avaliação estatística adicional:



In [ ]:
# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(testY.argmax(axis=1), predIdxs)

total = sum(sum(cm))

acc = (cm[0, 0] + cm[1, 1] + cm[2, 2]) / total
# sensitivity = (cm[0, 0] / (cm[0, 0]) + cm[0, 1])
# specificity = (cm[1, 1] / (cm[1, 0]) + cm[1, 1])
# show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("Total imagens para teste : {0}".format(total))
print("acc: {:.4f}".format(acc))
# print("sensitivity: {:.4f}".format(sensitivity))
# print("specificity: {:.4f}".format(specificity))

In [ ]:
{'covid', 'normal', 'pneumonia'}

In [ ]:
import seaborn as sns 
plt.style.use('fivethirtyeight')
matrix = confusion_matrix(predIdxs,testY.argmax(axis=1))
plt.figure(figsize=(8,8))
ax = plt.subplot()
sns.heatmap(matrix,annot=True,ax=ax)

diretorio_save ='{0}/{1}{2}.png'.format(diretorio_save_model_imagens,'Confusion Matrix',config.NomeModelo)

ax.set_xlabel('Predicted Labels',size=18)
ax.set_ylabel('True Labels',size=18)
ax.set_title('Confusion Matrix(0=Covid and 1=NonCovid and 2= pneumonia)',size=18)


plt.savefig(diretorio_save)


total = sum(sum(matrix))

acc = (matrix[0, 0] + matrix[1, 1] + matrix[2, 2]) / total
sensitivity = matrix[0, 0] / (matrix[0, 0] + cm[0, 1])
specificity = matrix[1, 1] / (matrix[1, 0] + cm[1, 1])


print("acc: {:.4f}".format(acc))
# print("sensitivity: {:.4f}".format(sensitivity))
# print("specificity: {:.4f}".format(specificity))


## Em seguida, plotamos nosso histórico de precisão / perda de treinamento para inspeção, produzindo o plot para um arquivo de imagem:

In [ ]:

config.to_dict()



In [ ]:
# traçar a perda e a precisão do treinamento
# plot the training loss and accuracy

diretorio_saveplot ='{0}/{1}_plot.png'.format(diretorio_save_model_imagens,config.NomeModelo)

N = config.EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(diretorio_saveplot)

## Agora Salvamos o modelo 

In [ ]:
diretorio_save_model_imagens

In [ ]:
# serialize the model to disk
model.save('{0}/{1}'.format(diretorio_save_model_imagens,config.NomeModelo))

## Validação de novas imagens

In [ ]:
output_model =r'{0}\{1}'.format(diretorio_raiz,'base_conhecimento')

In [ ]:
from tensorflow.keras.models import load_model
# Recreate the exact same model, including its weights and the optimizer
new_model = load_model('{0}/{1}'.format(diretorio_save_model_imagens,config.NomeModelo))

# Show the model architecture
new_model.summary()

## informações do modelo selecionado


In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = new_model.predict(testX, batch_size=config.BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs, target_names=lb.classes_))

## Teste de imagem com Covid


In [ ]:
def getArrayFromImage(var_imagem):
    test_image = cv2.imread(var_imagem)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
    test_image = cv2.resize(test_image, (224, 224))
    imagem_array = np.expand_dims(test_image, axis = 0)

    return imagem_array,test_image


In [ ]:
import random as r
var_imagem = r.choice(lista_imagens_com_Covid)

imagem_array,image  = getArrayFromImage(var_imagem)


In [ ]:
var_imagem

In [ ]:
{'covid', 'normal', 'pneumonia'}

## executando a predição de uma nova imagem

In [ ]:

pred = new_model.predict(imagem_array)
print(pred)

In [ ]:




print(np.argmax(pred, axis=1)[0])
if pred[0][0] == 1:
  plt.title('Prediction: Covid-19')
elif pred[0][1] == 1:
  plt.title('Prediction: normal')
elif pred[0][2] == 1:
  plt.title('Prediction: pneumonia')
      
plt.imshow(image)
    






In [ ]:
labels=['covid','normal','pneumonia']
plt.figure(figsize =(40,40))
for i in range(36):
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.title(labels[(np.argmax(testY[i], axis=0))])
    plt.imshow(trainX.reshape(-1,224,224,3)[i])
    
    if(predIdxs[i]==(np.argmax(testY[i], axis=0))):
        plt.xlabel(labels[predIdxs[i]],color='blue')
    else:
        plt.xlabel(labels[predIdxs[i]],color='red')
plt.show()

## Diagnóstico automático de COVID-19 a partir de resultados de imagens de raios-X

### Como você pode ver nos resultados acima, nosso detector COVID-19 automático está obtendo ~ 90-92% de precisão em nosso conjunto de dados de amostra com base apenas em imagens de raios-X  - nenhum outro dado, incluindo localização geográfica, densidade populacional etc. foi usado para treinar esse modelo.


### Também estamos obtendo 100% de sensibilidade e 80% de especificidade, o que implica que:

### Dos pacientes que fazem tem COVID-19 (ou seja, verdadeiros positivos), poderíamos identificá-los com precisão como “COVID-19 positiva” 100% do tempo , usando o nosso modelo.

### Dos pacientes que não têm COVID-19 (isto é, verdadeiros negativos), poderíamos identificá-los com precisão como "COVID-19 negativo" apenas 80% do tempo usando nosso modelo.

## Ser capaz de detectar com precisão o COVID-19 com 100% de precisão é ótimo; no entanto, nossa verdadeira taxa negativa é um pouco preocupante  - não queremos classificar alguém como "COVID-19 negativo" quando é "COVID-19 positivo".

## De fato, a última coisa que queremos fazer é dizer a um paciente que eles são negativos para COVID-19 e depois levá-los para casa e infectar sua família e amigos; transmitindo assim a doença ainda mais.

## Também queremos ter muito cuidado com nossa taxa de falsos positivos  - não queremos classificar erroneamente alguém como "positivo para COVID-19" , colocá-lo em quarentena com outros pacientes positivos para COVID-19 e infectar uma pessoa que nunca teve realmente o problema. vírus.

## Equilibrar sensibilidade e especificidade é incrivelmente desafiador quando se trata de aplicações médicas, especialmente doenças infecciosas que podem ser transmitidas rapidamente, como o COVID-19.

# Limitações


In [ ]:
Uma das maiores limitações do método discutido neste tutorial são os dados.

Simplesmente não temos dados suficientes (confiáveis) para treinar um detector COVID-19.

Os hospitais já estão sobrecarregados com o número de casos COVID-19 e, dados os direitos e a confidencialidade dos pacientes, fica ainda mais difícil montar conjuntos de dados de imagens médicas de qualidade em tempo hábil.

Imagino que nos próximos 12 a 18 meses teremos mais conjuntos de dados de imagem COVID-19 de alta qualidade; mas, por enquanto, só podemos nos contentar com o que temos.